In [8]:
import sys
sys.path.append("../")
sys.path.append("../src")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

testing are done in `test.py`

In [9]:
result = pd.read_csv("./test_results.csv")

Interpretation of Columns
- `similarity_response_answer`: cosine similarity between response and manually labeled answer
- `similarity_response_question`: cosine similarity between response and input question

In [10]:
result.head()

,Question,model,parser,similarity_response_question,similarity_response_answer
0,Are investment theses predicated on the contin...,llama2,llamaparse,0.774616,0.968329
1,Are investment theses predicated on the contin...,llama2,pdfplumber,0.863810,0.799212
2,Are investment theses predicated on the contin...,llama2,pypdfloader,0.859502,0.819184
3,Are investment theses predicated on the contin...,llama2,unstructured,0.854410,0.818009
4,Are investment theses predicated on the contin...,nlp_base,llamaparse,0.728865,0.784294


In [11]:
result.iloc[:, 1:].groupby(["model", "parser"]).mean()

similarity_response_question  \
model         parser                                       
llama2        llamaparse                        0.844584   
              pdfplumber                        0.850553   
              pypdfloader                       0.856681   
              unstructured                      0.848088   
nlp_base      llamaparse                        0.745874   
              pdfplumber                        0.759380   
              pypdfloader                       0.741713   
              unstructured                      0.906865   
nlp_langchain llamaparse                        0.848223   
              pdfplumber                        0.759380   
              pypdfloader                       0.741713   
              unstructured                      0.906865   

                            similarity_response_answer  
model         parser                                    
llama2        llamaparse                      0.830534  
              pdfplumber                      0.815374  
              pypdfloader                     0.819354  
              unstructured                    0.797751  
nlp_base      llamaparse                      0.791079  
              pdfplumber                      0.806773  
              pypdfloader                     0.792598  
              unstructured                    0.740309  
nlp_langchain llamaparse                      0.837659  
              pdfplumber                      0.776838  
              pypdfloader                     0.788849  
              unstructured                    0.839705

similarity: repsonese vs question

In [12]:
pd.pivot_table(result, index="model", columns="parser", values="similarity_response_question")

parser,llamaparse,pdfplumber,pypdfloader,unstructured
model,,,,
llama2,0.844584,0.850553,0.856681,0.848088
nlp_base,0.745874,0.759380,0.741713,0.906865
nlp_langchain,0.848223,0.759380,0.741713,0.906865


similarity: responese vs labeled answer

In [13]:
pd.pivot_table(result, index="model", columns="parser", values="similarity_response_answer")

parser,llamaparse,pdfplumber,pypdfloader,unstructured
model,,,,
llama2,0.830534,0.815374,0.819354,0.797751
nlp_base,0.791079,0.806773,0.792598,0.740309
nlp_langchain,0.837659,0.776838,0.788849,0.839705
